<a href="https://colab.research.google.com/github/totminaekaterina/NLP/blob/main/meatinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
# Шаг 1: Подготовка данных
data = pd.read_csv('/content/meatinfo.csv', encoding='utf-8-sig', sep=';')

data = data.dropna()  # Удаление строк с пропущенными значениями
texts = data['text'].tolist()
labels = data['type'].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Шаг 2: Предварительная обработка текста
def preprocess_text(text):
    if isinstance(text, float):  # Проверка на пропущенное значение (тип float)
        text = str(text)  # Преобразование в строку
    text = text.lower()  # Приведение к нижнему регистру
    text = text.strip()  # Удаление лишних пробелов
    return text

train_texts = [preprocess_text(text) for text in train_texts]
test_texts = [preprocess_text(text) for text in test_texts]

In [ ]:
# Шаг 3: Создание словаря
word_counts = Counter(' '.join(train_texts).split())
vocabulary = sorted(word_counts, key=word_counts.get, reverse=True)
word_to_index = {word: index + 1 for index, word in enumerate(vocabulary)}  # Используем индекс 0 для заполнения (padding)

In [ ]:
vocabulary[:5]

['в', 'цена', 'руб.', 'говядина', 'и']

In [ ]:
# Шаг 4: Преобразование данных
def encode_text(text):
    return [word_to_index.get(word, 0) for word in text.split()]  # 0 - индекс для неизвестных слов

train_encoded = [encode_text(text) for text in train_texts]
test_encoded = [encode_text(text) for text in test_texts]

In [ ]:
# Шаг 5: Обучение модели
max_sequence_length = max(len(seq) for seq in train_encoded + test_encoded)

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])  # Используем только последний выход из RNN
        return output

input_size = len(word_to_index) + 1  # Размер словаря плюс 1 для заполнения (padding)
hidden_size = 128
output_size = len(set(labels))
model = RNNClassifier(input_size, hidden_size, output_size)
model.to(device)

# Шаг 6 (продолжение): Обучение модели
batch_size = 1
num_epochs = 10
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Функция для применения паддинга к пакету текстов
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for seq in sequences:
        if len(seq) < max_length:
            padded_seq = seq + [0] * (max_length - len(seq))  # Используем заполнитель 0
        else:
            padded_seq = seq[:max_length]
        padded_sequences.append(padded_seq)
    return padded_sequences

# Создание словаря для преобразования меток в числовой формат
label_to_index = {}
index_to_label = {}
for i, label in enumerate(set(labels)):
    label_to_index[label] = i
    index_to_label[i] = label

# Преобразование меток в числовой формат
train_labels = [label_to_index[label] for label in train_labels]
test_labels = [label_to_index[label] for label in test_labels]

train_data = list(zip(train_encoded, train_labels))
num_batches = len(train_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        batch_data = train_data[batch * batch_size: (batch + 1) * batch_size]
        batch_texts, batch_labels = zip(*batch_data)

        optimizer.zero_grad()

        # Применение паддинга к пакету текстов
        batch_texts = pad_sequences(batch_texts, max_sequence_length)

        batch_texts = torch.LongTensor(batch_texts).to(device)
        batch_labels = torch.LongTensor(batch_labels).to(device)

        outputs = model(batch_texts)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Шаг 7: Оценка модели
test_data = list(zip(test_encoded, test_labels))
test_texts, test_labels = zip(*test_data)
test_texts = pad_sequences(test_texts, max_sequence_length)

test_texts = torch.LongTensor(test_texts).to(device)
test_labels = torch.LongTensor(test_labels).to(device)

model.eval()
with torch.no_grad():
    test_outputs = model(test_texts)

_, predicted_labels = torch.max(test_outputs, 1)
accuracy = (predicted_labels == test_labels).sum().item() / len(test_labels)
print(f'Test Accuracy: {accuracy}')


In [ ]:
# Преобразование новых текстов в числовой формат
new_texts = [
    'Набор для бульона свиной',
    'Набор для бульона свиной, в наличии, 76р/кг.',
    'Мясо премиум Предлагаем котлетное мясо мраморной говядины.',
    'спинка цб'
]
new_encoded = [encode_text(text) for text in new_texts]

# Применение паддинга к новым текстам
new_texts_padded = pad_sequences(new_encoded, max_sequence_length)

# Преобразование в тензор и перенос на GPU
new_texts_tensor = torch.LongTensor(new_texts_padded).to(device)

# Вычисление предсказаний для новых текстов
model.eval()
with torch.no_grad():
    new_outputs = model(new_texts_tensor)

# Преобразование индексов предсказаний в метки классов
predicted_labels = [index_to_label[index.item()] for index in torch.argmax(new_outputs, dim=1)]

# Вывод результатов
for text, label in zip(new_texts, predicted_labels):
    print(f'input: {text}')
    print(f'output: {label}')
    print()
